In [11]:
import sys
sys.path.append("../../../MVCNN")
import torch
from tools.ImgDataset import MultiviewImgDataset
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn import functional as F
import random
from torch.utils.data import Subset

In [2]:
pruned_models = {
    0.0: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.00.pth", map_location=torch.device('mps')),
    0.1: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.10.pth", map_location=torch.device('mps')),
    0.3: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.30.pth", map_location=torch.device('mps')),
    0.5: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.50.pth", map_location=torch.device('mps')),
    # 0.7: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.70.pth", map_location=torch.device('mps')),
    0.9: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.90.pth", map_location=torch.device('mps')),
}

In [3]:
test_path = "../../../MVCNN/ModelNet40-12View/*/test"
train_path = "../../../MVCNN/ModelNet40-12View/*/train"
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
dataset = MultiviewImgDataset(
    root_dir=train_path,
    scale_aug=False,
    num_models=0,
    num_views=12,
)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)
print(f"Samples in dataset: {len(dataset)}")

Samples in dataset: 7589


In [12]:
num_samples = 100

In [13]:
dataset_indices = random.sample(range(len(dataset)), num_samples)
dataset = Subset(dataset, dataset_indices)

In [5]:
print(len(dataloader.dataset[0][1]))

12


In [ ]:
def get_accuracy(model):
    all_correct = 0
    all_samples = 0
    
    wrong_class = np.zeros(33, dtype=int)
    samples_class = np.zeros(33, dtype=int)
    
    model.eval()
    model = model.to(device)
    with torch.no_grad():
        pbar = tqdm(dataloader, total=len(dataloader), desc="Testing")
        for _, data in enumerate(pbar):
            labels, images = data[0], data[1]
            labels = labels.to(device)
            images = images.to(device)       
            N, V, C, H, W = images.size()
            
            x = images.view(-1, C, H, W)
            net_1_out = model.net_1(x)
            y = F.adaptive_avg_pool2d(net_1_out, (7, 7))
            y = y.view(N, V, -1)
            y = torch.max(y, dim=1)[0]
            output = model.net_2(y)
            preds = output.argmax(dim=1)
            break
            
            batch_correct = (preds == labels).sum().item()
            all_correct += batch_correct
            all_samples += N
            
            for i in range(N):
                if preds[i] != labels[i]:
                    wrong_class[labels[i].cpu().item()] += 1
                samples_class[labels[i].cpu().item()] += 1

        overall_acc = all_correct / all_samples
        per_class_acc = (samples_class - wrong_class) / np.maximum(samples_class, 1)
        mean_class = per_class_acc[samples_class > 0].mean()

        print(f"Overall accuracy: {overall_acc:.4f}")
        print(f"Mean class accuracy: {mean_class:.4f}")

In [9]:
for key in pruned_models.keys():
    model = pruned_models[key]
    print(f"Testing model with pruning rate {key}")
    get_accuracy(model)
    print("-" * 50)
    print()

Testing model with pruning rate 0.0


Testing:   0%|          | 0/633 [00:00<?, ?it/s]

torch.Size([144, 3, 224, 224])
torch.Size([144, 512, 7, 7])
torch.Size([144, 512, 7, 7])
torch.Size([12, 12, 25088])


Testing:   0%|          | 0/633 [00:09<?, ?it/s]


ZeroDivisionError: division by zero

In [ ]:
random_img = torch.randn(1, 12, 3, 224, 224).to(device)
N, V, C, H, W = random_img.size() 

In [ ]:
out = pruned_models[0.5].net_1(random_img.view(-1, C, H, W))

In [ ]:
y = torch.functional.F.adaptive_avg_pool2d(out, (7, 7)).view(12, -1)

In [ ]:
y.shape

torch.Size([12, 25088])

In [ ]:
y = torch.max(y, 0)[0].unsqueeze(0)
y.shape

torch.Size([1, 25088])

In [ ]:
model.net_2(y)

tensor([[-1236.4639,  -684.7574,  -813.0821,  -749.7675, -1122.9403, -1392.8016,
         -1348.2399,  -832.9878,  -868.5643,  -896.7075,  -859.9577, -1069.5975,
          -739.2985,  -630.5588, -1140.3524, -1270.0352,  -857.8713,  -645.1979,
          -547.7659, -1266.9254,  -717.3286,  -948.5233,  -761.9027,  -692.7169,
          -828.7469,  -826.9893,  -998.6613,  -737.7090,  -755.4247,  -792.4379,
          -895.1537,  -965.6391,  -655.9093]], device='mps:0',
       grad_fn=<LinearBackward0>)